In [1]:
from PIL import Image, ImageOps
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
data = []

2023-11-27 16:44:37.367028: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.2 when it was built against 1.14.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
def one_hot(num):
    arr = np.zeros(3)
    arr[num] = 1
    return arr

## Load Images


In [3]:
def get_from_path(path, num):
    p_dir = os.listdir(path)
    for file in p_dir:
        img = Image.open(path + '/' + file)
        img = ImageOps.grayscale(img)
        img = img.resize((224, 224)) 
       
        img = np.array(img)
        img = img.reshape((224, 224, 1))
        if num == 1 and 'bacteria' in file:
            num = 2
        data.append((tf.convert_to_tensor(one_hot(num)), tf.convert_to_tensor(img)))
  

In [7]:
path = './chest_xray/train/NORMAL'
get_from_path(path, 0)
path = './chest_xray/train/PNEUMONIA'   
get_from_path(path, 1)
  

In [8]:
label, images = zip(*data)
label = list(label[:])
images = list(images[:])
img_data = []
for img in images:
    if len(img.shape) < 3:
        print(img.shape)
        img_data.append(np.expand_dims(img, axis=-1))
    else:
        img_data.append(img)

In [9]:
for img in img_data:
    print(img.shape)


(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 224, 1)
(224, 

In [10]:
data_set = tf.data.Dataset.from_tensor_slices((images, label))

In [11]:
data_set = data_set.shuffle(2048).batch(128)

## Create the Model


In [12]:
def make_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(10, (8, 8), input_shape=(224, 224, 1), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(20, (8, 8), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3)))
    model.add(keras.layers.Conv2D(30, (4, 4), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(500, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dense(3, activation='softmax'))
    return model

In [13]:

model = make_model()
model.compile(
    optimizer='adam',
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [14]:
model.fit(data_set, epochs=10)

Epoch 1/10
58/58 [==============================] - 67s 1s/step - loss: 0.8020 - accuracy: 0.8165
Epoch 2/10
58/58 [==============================] - 67s 1s/step - loss: 0.1585 - accuracy: 0.9398
Epoch 3/10
58/58 [==============================] - 72s 1s/step - loss: 0.1007 - accuracy: 0.9605
Epoch 4/10
58/58 [==============================] - 74s 1s/step - loss: 0.0698 - accuracy: 0.9748
Epoch 5/10
58/58 [==============================] - 75s 1s/step - loss: 0.0641 - accuracy: 0.9755
Epoch 6/10
58/58 [==============================] - 73s 1s/step - loss: 0.0503 - accuracy: 0.9808
Epoch 7/10
58/58 [==============================] - 74s 1s/step - loss: 0.0442 - accuracy: 0.9834
Epoch 8/10
58/58 [==============================] - 78s 1s/step - loss: 0.0374 - accuracy: 0.9862
Epoch 9/10
58/58 [==============================] - 73s 1s/step - loss: 0.0630 - accuracy: 0.9781
Epoch 10/10
58/58 [==============================] - 70s 1s/step - loss: 0.0343 - accuracy: 0.9871


In [15]:
model.save_weights('model.tf')

In [ ]:
model.load_weights('model.tf')

In [16]:
data = []

In [17]:
path = './chest_xray/test/NORMAL'
get_from_path(path, 0)
path = './chest_xray/test/PNEUMONIA'   
get_from_path(path, 1)

## Test the Model

In [18]:
data_set_test = tf.data.Dataset.from_tensor_slices((images, label))
data_set_test = data_set.shuffle(2048)

In [19]:
loss, acc = model.evaluate(data_set_test)
print(f'Accuracy: {acc}')

58/58 [==============================] - 18s 310ms/step - loss: 0.0296 - accuracy: 0.9911
Accuracy: 0.9911056160926819
